In [1]:
import os
import csv
import math
import wandb
import numpy as np
from pathlib import Path 

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from wandb.keras import WandbCallback
from keras.layers import Dense, Flatten, Dropout

Using TensorFlow backend.


In [2]:
#Getting the latest file from the folder
record_path = Path("RecordSummary")
training_data_list = list(filter(os.path.isfile,record_path.glob('*.csv')))
training_data_list.sort(key=lambda x: os.path.getmtime(x))
training_data = training_data_list[-1]

x_train_percent = 60
x_test_percent = 100-x_train_percent
x_train = []
x_test = []
y_train = []
y_test = []

with open(training_data) as csv_file:
    data = list(csv.reader(csv_file))
    lines = list(map(list, zip(*data)))

nr_x_train = math.ceil(len(lines[0])*x_train_percent/100)
lines = np.asarray(lines)

#X stands for input, Y stands for output

x_train = lines[1:, 0:nr_x_train]
y_train = lines[0, 0:nr_x_train]

x_test = lines[1:, nr_x_train:] 
y_test = lines[0, nr_x_train:]
x_train = x_train.transpose()
x_test = x_test.transpose()

x_train = [a.reshape(4,96) for a in x_train]
x_test = [a.reshape(4,96) for a in x_test]

is_up_y_train = y_train=='up'
is_up_y_test = y_test=='up'
labels = ["Down", "Up"]

In [3]:
channels = np.shape(x_train)[1]
samples = np.shape(x_train)[2]

print(channels)
print(samples)

4
96


In [4]:
print(np.shape(x_train))
print(np.shape(is_up_y_train))
print(np.shape(x_test))
print(np.shape(is_up_y_test))

(130, 4, 96)
(130,)
(86, 4, 96)
(86,)


In [5]:
print(type(x_train))
print(type(is_up_y_train))

<class 'list'>
<class 'numpy.ndarray'>


In [6]:
#converting <class 'list'> to <class 'numpy.ndarray'>
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

In [7]:
#Normalizing data:
x_train = x_train.astype("float")
x_test = x_test.astype("float")
merge = np.vstack((x_train, x_test))
minval = np.amin(merge)
maxval = np.amax(merge)
print(f'Min: {minval}, Max:{maxval}')
x_train = (x_train-minval)/(maxval-minval)
x_test = (x_test-minval)/(maxval-minval)

Min: -194.824, Max:221.68


In [8]:
# create model
model=Sequential()
model.add(Flatten(input_shape=(channels,samples)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam',
                metrics=['accuracy'])

In [9]:
# logging code
run = wandb.init(project="eeg-up-down")
config = run.config
config.epochs = 1000
# Fit the model
model.fit(x_train, is_up_y_train, epochs=config.epochs, validation_data=(x_test, is_up_y_test),
                    callbacks=[WandbCallback(labels=labels)])

Train on 130 samples, validate on 86 samples
Epoch 1/100
130/130 [==============================] - 0s 488us/step - loss: 0.2556 - accuracy: 0.5000 - val_loss: 0.2360 - val_accuracy: 0.6163
Epoch 2/100
130/130 [==============================] - 0s 52us/step - loss: 0.2573 - accuracy: 0.5462 - val_loss: 0.2361 - val_accuracy: 0.6163
Epoch 3/100
130/130 [==============================] - 0s 53us/step - loss: 0.2501 - accuracy: 0.5538 - val_loss: 0.2474 - val_accuracy: 0.5698
Epoch 4/100
130/130 [==============================] - 0s 53us/step - loss: 0.2510 - accuracy: 0.4385 - val_loss: 0.2483 - val_accuracy: 0.5698
Epoch 5/100
130/130 [==============================] - 0s 60us/step - loss: 0.2495 - accuracy: 0.5231 - val_loss: 0.2393 - val_accuracy: 0.6163
Epoch 6/100
130/130 [==============================] - 0s 53us/step - loss: 0.2462 - accuracy: 0.5462 - val_loss: 0.2361 - val_accuracy: 0.6163
Epoch 7/100
130/130 [==============================] - 0s 45us/step - loss: 0.2540 - accur

130/130 [==============================] - 0s 60us/step - loss: 0.2453 - accuracy: 0.5462 - val_loss: 0.2364 - val_accuracy: 0.6163
Epoch 58/100
130/130 [==============================] - 0s 60us/step - loss: 0.2465 - accuracy: 0.5462 - val_loss: 0.2365 - val_accuracy: 0.6163
Epoch 59/100
130/130 [==============================] - 0s 45us/step - loss: 0.2452 - accuracy: 0.5692 - val_loss: 0.2454 - val_accuracy: 0.6628
Epoch 60/100
130/130 [==============================] - 0s 53us/step - loss: 0.2441 - accuracy: 0.6077 - val_loss: 0.2504 - val_accuracy: 0.4070
Epoch 61/100
130/130 [==============================] - 0s 75us/step - loss: 0.2456 - accuracy: 0.5538 - val_loss: 0.2444 - val_accuracy: 0.6279
Epoch 62/100
130/130 [==============================] - 0s 60us/step - loss: 0.2438 - accuracy: 0.5923 - val_loss: 0.2361 - val_accuracy: 0.6163
Epoch 63/100
130/130 [==============================] - 0s 52us/step - loss: 0.2499 - accuracy: 0.5462 - val_loss: 0.2367 - val_accuracy: 0.616

In [ ]:
print(model.predict(x_test[10:50,:,:]))
print(is_up_y_test[10:50])